In [1]:
%pylab inline

import pandas as pd
import numpy as np

from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
#from sklearn.metrics import roc_curve, auc, classification_report, confusion_matrix, roc_auc_score
from sklearn.metrics import mean_absolute_error

import datetime, time

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Read data
wnv_train = pd.read_csv('../working/train_f.csv', index_col=0)
wnv_test = pd.read_csv('../working/test_f.csv', index_col=0)

In [3]:
labels = wnv_train.NumMosquitos.values.astype('int')
yr_train = labels
wnv_train.drop('NumMosquitos', axis=1, inplace=True)
wnv_train.drop('WnvPresent', axis=1, inplace=True)
f_names = wnv_train.columns

In [13]:
# raw data
#yr_train = wnv_train[:,1].astype(int)
Xr_train =(wnv_train.values).astype('float64')
Xr_test =(wnv_test.values).astype('float64')

# normalize data
scaler = StandardScaler()
scaler.fit(Xr_train)
Xn_train = scaler.transform(Xr_train)
Xn_test = scaler.transform(Xr_test)

X_train, X_test, y_train, y_test = train_test_split(Xn_train, yr_train)

ValueError: operands could not be broadcast together with shapes (116293,518) (517,) (116293,518) 

In [5]:
clf = LogisticRegression()
clf.fit(X_train, y_train)

# use the model to predict the labels of the test data
predicted = clf.predict(X_test)
#proba = clf.predict_proba(X_test)
expected = y_test

In [6]:
mean_absolute_error(y_test, predicted)

10.030452988199468

In [14]:
wnv_test['NumMosquitos'] = clf.predict(Xn_test)

In [17]:
wnv_test['NumMosquitos']

0     50
1     50
2     50
3     50
4     50
5     50
6     50
7      7
8     50
9     50
10    50
11    50
12    50
13    50
14    50
...
116278    1
116279    1
116280    1
116281    1
116282    1
116283    1
116284    7
116285    1
116286    1
116287    1
116288    1
116289    1
116290    1
116291    1
116292    7
Name: NumMosquitos, Length: 116293, dtype: int32

In [18]:
wnv_test.to_csv('../working/test_fn.csv')